In [14]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import itertools
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import time

import warnings
warnings.filterwarnings('ignore')

# Get current working directory and go up one level to project root
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.llm_application import LLMAccountant, row2doc, doc2lancamento

In [15]:
df = pd.read_csv(PROJECT_ROOT / "data" / "input_com_categorias.csv")

In [16]:
dev_df, holdout_df = train_test_split(df, test_size=0.3, random_state=42)

In [17]:
k_range = range(3, 15)
threshold_range = [i/10 for i in range(5, 10)]
grid = list(itertools.product(k_range, threshold_range))

### O _cross-validation_ é apenas uma prova de conceito. A base de dados é muito pequena para apresentar alguma significância.

In [18]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracy_estimates = []
for k, threshold in tqdm(grid):
    accuracy_buffer = []
    for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(dev_df, dev_df['Conta Contábil'])):
        train_fold = dev_df.iloc[train_idx]
        val_fold = dev_df.iloc[val_idx]

        accountant = LLMAccountant(threshold=threshold, k=k)
        train_docs = [row2doc(row) for row in train_fold.iterrows()]
        accountant.add_source_of_truth(train_docs)

        val_docs = [row2doc(row) for row in val_fold.iterrows()]
        val_lancamentos = [doc2lancamento(doc) for doc in val_docs]

        output = accountant.batch(val_lancamentos)

        y_pred = [out['category'].category for out in output]
        y = val_fold['Conta Contábil']
        accuracy_buffer.append(accuracy_score(y, y_pred))
        
    accuracy_estimates.append(np.mean(accuracy_buffer))

  2%|▏         | 1/60 [00:50<49:34, 50.42s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-uJfYkIF00NcMUgE1YL2fDqbl on tokens per min (TPM): Limit 200000, Used 200000, Requested 1649. Please try again in 494ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
best_params = grid[np.argmax(accuracy_estimates)]
best_params

In [ ]:
best_accountant = LLMAccountant(threshold=0.7, k=10)
dev_docs = [row2doc(row) for row in dev_df.iterrows()]
accountant.add_source_of_truth(dev_docs)

In [ ]:
holdout_docs = [row2doc(row) for row in holdout_df.iterrows()]
holdout_lancamentos = [doc2lancamento(doc) for doc in holdout_docs]

output = accountant.batch(holdout_lancamentos)
y_pred = [out['category'].category for out in output]
y = holdout_df['Conta Contábil']

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(y, y_pred)

# Get unique labels for proper ordering
labels = sorted(list(set(y) | set(y_pred)))

# Create the plot
plt.figure(figsize=(10, 8))
sns.heatmap(cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=labels,
            yticklabels=labels,
            cbar_kws={'label': 'Count'})

plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print accuracy
from sklearn.metrics import accuracy_score
print(f"\nAccuracy: {accuracy_score(y, y_pred):.4f}")

Accuracy: 0.7027

# Oportunidades de melhoria:

### 1. Aqui foi selecionado o melhor modelo em termos de acurácia, mas isso não significa que este seja o melhor modelo para a aplicação. Devemos levar em conta que o LLM classifica melhor que o KNN na maioria dos casos, mas, ao mesmo tempo, apresenta um custo de execução ordens de magnitude mais alto. Uma futura aplicação dessa ideia deve considerar esse fator. Integrações com o `LangSmith` podem ajudar a encontrar um ótimo no problema do custo de geração das classes.

### 2. É possível melhorar o algoritmo KNN testando diferentes métricas de similaridade.

### 3. O suporte do sistema para contas recorrentes é limitado. A ideia é que o sistema seja capaz de identificar esse tipo de conta e classificá-las logicamente, em vez de utilizar a classificação por meio de modelos.